# Applications of Convex.jl in Optimization Involving Complex Numbers

## Ayush Pandey | JuliaCon 2017

[https://github.com/Ayush-iitkgp/JuliaCon2017Presentation](https://github.com/Ayush-iitkgp/JuliaCon2017Presentation)

<!--- http://www.damian.oquanta.info/posts/make-your-slides-with-ipython.html --->

# About Me 

* **BS and MS in Mathematics and Computing Sciences (July'12-June'17) at Indian Institute of Technology Kharagpur **

* **Google Summer of Code 2016 and 2017 student under the Julia Language**

* **GitHub:** [Ayush-iitkgp](https://github.com/ayush-iitkgp)

* **Website:** [https://ayush-iitkgp.github.io](https://ayush-iitkgp.github.io/) 



<!----#### Blogging my GSoC'16 experience at [http://ayush-iitkgp.rhcloud.com](http://ayush-iitkgp.rhcloud.com/) ---->

<!--- ## CVX.jl team

* [CVX.jl](https://github.com/cvxgrp/CVX.jl): Madeleine Udell, Karanveer Mohan, David Zeng, Jenny Hong
<!---* [ParallelSparseMatMul.jl](https://github.com/madeleineudell/ParallelSparseMatMul.jl): Madeleine Udell
--->

# Outline

* Power Flow Optimization
* Fidelity in Quantum Information
<!--* Benchmark -->

### Optimal Power Flow - Introduction

* Power flow study is an analysis of a connected electrical power system’s capability to adequately supply the connected load.  

![An example of a power network](Power_Network.png)

* **Unknowns: ** - Voltages angle and magnitude information for each bus
* **Knowns: ** - Load( such as appliances and lights.) and generator real power and voltage condition.
<!-- An electrical load is an electrical component or portion of a circuit that consumes (active) electric power. -->

### Optimal Power Flow - Mathematical Formulation

#### Constraints

![Each transmission line has four flows](Power_Network_2.png)

* $p_{ij}$: Active power entering the line from node i
* $q_{ij}$: Reactive power entering the line from node i
* Let $x_{i}$ denote the complex voltage for node i of the network. 

We have the following power balance equations which are **non-linear** in unknown $x_{i}$ and $x_{j}$.

![Power balance equations](Power_Network_3.png)

#### Objective 

Depends upon the business needs such as: 

* Minimize power losses in an electrical network
* Minimize cost of generation

### Optimal Power Flow - SDP Relaxation

* The original optimal power flow problem is non-convex in nature.
* Thanks to the [**lifting technique**](https://www.informs.org/content/download/320453/3031884/version/2/file/OStoday2016.pdf) which converts the above optimization problem to a SemiDefinite Programming Problem.
* The relaxed SDP problem finds the **near global solution** of the original non-convex problem.

### Example

* The data is taken from the IEEE 14 Bus test case which represents a portion of the American Electric Power System (in the Midwestern US) as of February, 1962.

In [37]:
using Convex  # Read the input data
using FactCheck
using MAT   #Pkg.add("MAT")
TOL = 1e-2;
input = matopen("Data.mat")
varnames = names(input)
Data = read(input, "inj","Y");
n=size(Data[2],1); # Create some intermediate variables
Y=Data[2];
inj=Data[1];

In [38]:
W = ComplexVariable(n,n); # W is the matrix of pairwise products of the voltages

objective = real(sum(diag(W))); # The objective is to minimize cost of generation

c1 = Constraint[]; # The constraints are power balance equations
for i=2:n
    push!(c1,sum(W[i,:].*(Y[i,:]'))==inj[i]);
end
c2 = W in :SDP;
c3 = real(W[1,1])==1.06^2;
push!(c1, c2);
push!(c1, c3);

In [39]:
p = maximize(objective,c1); # Create the problem
solve!(p); # Solve the problem
p.optval #15.125857662600703
evaluate(objective) #15.1258578588357

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 1344
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 393, constraints m = 812
Cones:	primal zero / dual free vars: 406
	sd vars: 406, sd blks: 1
Setup time: 6.53e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  3.16e-04 
   100| 3.92e-02  8.28e-02  6.88e-05 -1.43e+01 -1.43e+01  1.23e-16  1.01e-01 
   200| 7.31e-03  1.97e-02  1.66e-05 -1.48e+01 -1.48e+01  0.00e+00  1.61e-01 
   300| 2.71e-03  5.88e-03  5.13e-06 -1.50e

In [40]:
output = matopen("Res.mat"); # Verify the results
names(output);
outputData = read(output, "Wres");
Wres = outputData;
real_diff = real(W.value) - real(Wres);
imag_diff = imag(W.value) - imag(Wres);
@fact real_diff => roughly(zeros(n,n), TOL);
@fact imag_diff => roughly(zeros(n,n), TOL)

Success :: (line:441) :: fact was true
  Expression: imag_diff --> roughly(zeros(n,n),TOL)
    Expected: [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
    Occurred: [7.21891e-9 1.09908e-5 2.03142e-5 2.4322e-5 2.20655e-5 4.88765e-5 4.56099e-5 4.655

### Fidelity in Quantum Information Theory - Introduction

* This example is inspired from a lecture of John Watrous in the [course on Theory of Quantum Information](https://cs.uwaterloo.ca/~watrous/CS766/LectureNotes/08.pdf).

* Fidelity is a measure of the **closeness** of two quantum states.
   
* The ability to distinguish between the quantum states is equivalent to the ability to distinguish between the classical probability distributions. 

* If fidelity between two states is 1, they are the same quantum state.

<!-- If an experimenter is attempting to determine whether a quantum state is either of two possibilities {\displaystyle \rho } \rho  or {\displaystyle \sigma } \sigma , the most general possible measurement they can make on the state is a POVM, which is described by a set of Hermitian positive semidefinite operators {\displaystyle \{F_{i}\}} \{F_{i}\}. If the state given to the experimenter is {\displaystyle \rho } \rho , they will witness outcome {\displaystyle i} i with probability {\displaystyle p_{i}=\mathrm {Tr} [\rho F_{i}]} p_{i}={\mathrm  {Tr}}[\rho F_{i}], and likewise with probability {\displaystyle q_{i}=\mathrm {Tr} [\sigma F_{i}]} q_{i}={\mathrm  {Tr}}[\sigma F_{i}] for {\displaystyle \sigma } \sigma  -->

<!-- Wikipedia Link -->
<!-- https://en.wikipedia.org/wiki/Fidelity_of_quantum_states -->

* **Application** 

![Quantum Cryptography](Fidelity.png)

* The Fidelity between two Hermitian semidefinite matrices P and Q is defined as:

$$F(P,Q) = {||{P}^{1/2}{Q}^{1/2} ||}_{tr} =  \max\; |trace({P}^{1/2}U{Q}^{1/2})|$$

where the trace norm $||.||_{tr}$ is the sum of the singular values, and the maximization goes over the set of all unitary matrices U.

### Fidelity in Quantum Information Theory - Mathematical Formulation

Fidelity can be expressed as the optimal value of the following complex-valued SDP:

$$ \textbf{maximize} \frac{1}{2} trace(Z+Z^*)$$

$$\text{subject to } \left[\begin{array}{cc}P&Z\\{Z}^{*}&Q\end{array}\right] \succeq 0$$

$$\text{where } Z \in \mathbf {C}^{n \times n}$$

### Example

In [46]:
n = 20 # Create the data
P = randn(n,n) + im*randn(n,n);
P = P*P';
Q = randn(n,n) + im*randn(n,n);
Q = Q*Q';

Z = ComplexVariable(n,n); # Declare convex variable

objective = 0.5*real(trace(Z+Z'));  # Specify the problem
constraint = [P Z;Z' Q] ⪰ 0;
problem = maximize(objective,constraint);
 
solve!(problem) # Solve the problem

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 1621
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 801, constraints m = 6401
Cones:	primal zero / dual free vars: 3161
	sd vars: 3240, sd blks: 1
Setup time: 1.91e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf      -inf       inf  7.54e-03 
   100| 1.01e-03  2.31e-01  2.50e-02 -5.66e+02 -5.39e+02  0.00e+00  2.30e-01 
   200| 2.22e-04  5.11e-02  3.54e-03 -5.75e+02 -5.71e+02  0.00e+00  4.48e-01 
   300| 7.54e-05  1.64e-02  7.96e-04 -5.

In [47]:
# Verify that computer fidelity is equal to actual fidelity
computed_fidelity = evaluate(objective)
P1,P2 = eig(P);
sqP = P2 * diagm([p1^0.5 for p1 in P1]) * P2'
Q1,Q2 = eig(Q)
sqQ = Q2 * diagm([q1^0.5 for q1 in Q1]) * Q2'
actual_fidelity = sum(svd(sqP * sqQ)[2])
diff = computed_fidelity - actual_fidelity
@fact diff => roughly(0, TOL)

Success :: (line:441) :: fact was true
  Expression: diff --> roughly(0,TOL)
    Expected: 0
    Occurred: -0.0010774993736504257

# Thank You!